### ERIS CORBAXHI TASK 1 - SOLUTIONS

In [36]:
# Importing libraries

import pandas as pd

In [37]:
# Read the data file into Python as a pandas data frame

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Fall2024_FINC514/code/train.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# A. [10pt]
# Using Titanic train.csv dataset (same as HW1 and 2):
# Use the "survived" as output, and "pclass", "fare", "age", "sex" as input variables.
# Assuming we are running the analysis on "predicting whether a person will survive".
# Please do proper cleaning and data transformation (one-hot encoding) on variables if necessary. (we did it in HW2)

In [38]:
# Checking for missing values
for column in df.columns:
  missing_values = df[column].isnull().sum()
  if missing_values > 0:
    print(f"Variable '{column}' has {missing_values} missing values.")
  else:
    print(f"Variable '{column}' has no missing values.")

Variable 'PassengerId' has no missing values.
Variable 'Survived' has no missing values.
Variable 'Pclass' has no missing values.
Variable 'Name' has no missing values.
Variable 'Sex' has no missing values.
Variable 'Age' has 177 missing values.
Variable 'SibSp' has no missing values.
Variable 'Parch' has no missing values.
Variable 'Ticket' has no missing values.
Variable 'Fare' has no missing values.
Variable 'Cabin' has 687 missing values.
Variable 'Embarked' has 2 missing values.


In [39]:
# Creating a new DataFrame df_analysis by selecting requested columns from the existing DataFrame df.

df_analysis = df[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']]

In [40]:
for column in df_analysis.columns:
  missing_values = df_analysis[column].isnull().sum()
  if missing_values > 0:
    print(f"Variable '{column}' has {missing_values} missing values.")
  else:
    print(f"Variable '{column}' has no missing values.")

Variable 'Survived' has no missing values.
Variable 'Pclass' has no missing values.
Variable 'Sex' has no missing values.
Variable 'Age' has 177 missing values.
Variable 'Fare' has no missing values.


In [41]:
# Creating a new DataFrame df_analysis by removing rows from the original DataFrame df_analysis that contain missing values in the Age column.
# df_analysis = df_analysis.dropna(subset=['Age'])

# Handling missing values
# Age: We will fill missing values with the avarage
df_analysis['Age'] = df_analysis['Age'].fillna(df_analysis['Age'].mean())

<ipython-input-41-a3c2b9034128>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis['Age'] = df_analysis['Age'].fillna(df_analysis['Age'].mean())


In [42]:
# Get a list of columns with data type 'object' (often used for categorical data)
categorical_cols = df_analysis.select_dtypes(include=['object']).columns.tolist()

# Print the list of categorical columns
print(f"Categorical columns in df_analysis: {categorical_cols}")

Categorical columns in df_analysis: ['Sex']


In [43]:
# Specifying which variable should be treated as categorical when performing various operations
categorical_variables = ['Sex']

# Apply one-hot encoding to categorical variables
df_analysis = pd.get_dummies(df_analysis, columns=categorical_variables, dummy_na=False)

In [44]:
print(df_analysis.head())

   Survived  Pclass   Age     Fare  Sex_female  Sex_male
0         0       3  22.0   7.2500       False      True
1         1       1  38.0  71.2833        True     False
2         1       3  26.0   7.9250        True     False
3         1       1  35.0  53.1000        True     False
4         0       3  35.0   8.0500       False      True


In [ ]:
# B. [50pt]
# Use the training/testing method on 3 models - logistic regression,decision tree, and SVM.
# (You can use 80% for training, and 20% for testing)
#[10pt] Train each model
#[15pt] Print the accuracy score for each model
#[5pt] Pick the best model based on accuray score of the testing dataset.

In [45]:
# Importing necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd


# Splitting the dataset into features and target variable
X = df_analysis[['Pclass', 'Age', 'Fare', 'Sex_female', 'Sex_male']]
y = df_analysis['Survived']

# Splitting the dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



Logistic Regression

In [46]:
# Initializing models
log_reg = LogisticRegression()

# Training the models
log_reg.fit(X_train, y_train)

# Making predictions
log_reg_pred = log_reg.predict(X_test)

# Calculating accuracy scores
log_reg_acc = accuracy_score(y_test, log_reg_pred)

# Printing accuracy scores for each model
log_reg_acc

0.8044692737430168

Decision Tree

In [47]:
# Applying Decision tree model
# Initializing models
dec_tree = DecisionTreeClassifier()

# Training the models
dec_tree.fit(X_train, y_train)

# Making predictions
dec_tree_pred = dec_tree.predict(X_test)

# Calculating accuracy scores
dec_tree_acc = accuracy_score(y_test, dec_tree_pred)

# Printing accuracy scores for each model
dec_tree_acc

0.7374301675977654

SVM (Support Vector Machines)

In [48]:
# Initializing models
svm_model = SVC()

# Training the models
svm_model.fit(X_train, y_train)

# Making predictions
svm_pred = svm_model.predict(X_test)

# Calculating accuracy scores
svm_acc = accuracy_score(y_test, svm_pred)

# Printing accuracy scores for each model
svm_acc

0.6536312849162011

In [ ]:
# The best regression is the Logistic Regression model as it provides the highest value of accuracy

Cross validation with 6 folds on 3 models

In [ ]:
# C.
#[20pt] Use cross validation with 6 folds on 3 models - logistic regression,decision tree, and SVM.
#[10pt] Train each model using cross validation.
#[15pt] Get the accuracy score for each model
#[5pt] Pick the best model

In [49]:
# Importing necessary libraries

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Select relevant columns for the model
features = ['Pclass', 'Age', 'Fare', 'Sex']
target = 'Survived'


In [50]:
# Preprocessing: one-hot encoding for categorical features and handling missing values
numeric_features = ['Pclass', 'Age', 'Fare']
categorical_features = ['Sex']

# Build preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(drop='first'))]), categorical_features)
    ])

In [51]:
# Prepare models for evaluation
models = {
    "Logistic Regression": LogisticRegression(max_iter=200),
    "Decision Tree": DecisionTreeClassifier(),
    "SVM": SVC(),
}

# Initialize cross-validation
kf = KFold(n_splits=6, shuffle=True, random_state=42)

# Store results for comparison
results = {}

# Perform cross-validation on each model
for model_name, model in models.items():
    # Create pipeline with preprocessing and model
    clf = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])

    # Perform cross-validation and store accuracy scores
    cv_results = cross_val_score(clf, df[features], df[target], cv=kf, scoring='accuracy')
    results[model_name] = cv_results.mean()


In [52]:
# Find the best model based on accuracy
best_model = max(results, key=results.get)

# Display the results
print("Model Evaluation Results:")
for model, accuracy in results.items():
    print(f"{model}: {accuracy:.4f}")

print(f"\nBest Model: {best_model}")

Model Evaluation Results:
Logistic Regression: 0.7789
Decision Tree: 0.7711
SVM: 0.6733

Best Model: Logistic Regression


In [ ]:
# The best regression is the Logistic Regression model as it provides the highest value of accuracy

### ERIS CORBAXHI TASK 2 - SOLUTIONS

In [ ]:
# [20pt]: Regression Model Selection

# In the last homework, we built a linear regression model
# using Fare as dependent varaible, and Survived, Pclass, Sex, Age, Embarked as independent variables.
# Please run this model again and get mean squared error.
# Run the model by removing 1 or 2 variables (you pick which to remove), and see the mean squared error.
# Is removing the variables improve your model performance?

In [53]:
# In the last homework, we built a linear regression model
# using Fare as dependent varaible, and Survived, Pclass, Sex, Age, Embarked as independent variables.df = pd.read_csv('/content/drive/MyDrive/Fall2024_FINC514/code/train.csv')

df_analysis = df[['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked']]

df_analysis = df_analysis.dropna(subset=['Embarked'])

# Calculate the average age and fare
average_age = df_analysis['Age'].mean()

# Fill missing values in 'Age' with the average age
df_analysis = df_analysis.assign(Age=df_analysis['Age'].fillna(average_age))

# Identify categorical variables
categorical_variables = ['Sex', 'Embarked']

# Apply one-hot encoding to categorical variables
df_analysis = pd.get_dummies(df_analysis, columns=categorical_variables, dummy_na=False)

print(df_analysis)
print(f"Number of observations in the dataset: {len(df_analysis)}")

     Survived  Pclass        Age     Fare  Sex_female  Sex_male  Embarked_C  \
0           0       3  22.000000   7.2500       False      True       False   
1           1       1  38.000000  71.2833        True     False        True   
2           1       3  26.000000   7.9250        True     False       False   
3           1       1  35.000000  53.1000        True     False       False   
4           0       3  35.000000   8.0500       False      True       False   
..        ...     ...        ...      ...         ...       ...         ...   
886         0       2  27.000000  13.0000       False      True       False   
887         1       1  19.000000  30.0000        True     False       False   
888         0       3  29.642093  23.4500        True     False       False   
889         1       1  26.000000  30.0000       False      True        True   
890         0       3  32.000000   7.7500       False      True       False   

     Embarked_Q  Embarked_S  
0         False      

In [54]:
# Please run this model again and get mean squared error.

import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error

# Define the formula for the linear regression model
formula = 'Fare ~ Survived + Pclass + Age'

# Fit the linear regression model
model = sm.ols(formula, data=df_analysis).fit()

# Print the model summary
print(model.summary())

# Get the number of observations used in the analysis
n_observations = len(df_analysis)
print(f"Number of observations: {n_observations}")

# Calculate the predicted values
predicted = model.fittedvalues

# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(df_analysis['Fare'], predicted)

# Print the MSE
print(f"Mean Squared Error: {mse}")


                            OLS Regression Results                            
Dep. Variable:                   Fare   R-squared:                       0.313
Model:                            OLS   Adj. R-squared:                  0.311
Method:                 Least Squares   F-statistic:                     134.4
Date:                Wed, 16 Oct 2024   Prob (F-statistic):           9.06e-72
Time:                        16:52:20   Log-Likelihood:                -4566.4
No. Observations:                 889   AIC:                             9141.
Df Residuals:                     885   BIC:                             9160.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    116.4678      7.163     16.261      0.0

In [55]:
import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error

# Define the formula without 'Age' and 'Survived'
formula = 'Fare ~ Pclass'

# Fit the linear regression model
model_no_age_survived = sm.ols(formula, data=df_analysis).fit()

# Print the model summary
print(model_no_age_survived.summary())

# Calculate the predicted values
predicted_no_age_survived = model_no_age_survived.fittedvalues

# Calculate the Mean Squared Error (MSE) without 'Age' and 'Survived'
mse_no_age_survived = mean_squared_error(df_analysis['Fare'], predicted_no_age_survived)

# Print the MSE
print(f"Mean Squared Error (without 'Age' and 'Survived'): {mse_no_age_survived}")


                            OLS Regression Results                            
Dep. Variable:                   Fare   R-squared:                       0.301
Model:                            OLS   Adj. R-squared:                  0.300
Method:                 Least Squares   F-statistic:                     381.1
Date:                Wed, 16 Oct 2024   Prob (F-statistic):           7.03e-71
Time:                        16:52:25   Log-Likelihood:                -4574.5
No. Observations:                 889   AIC:                             9153.
Df Residuals:                     887   BIC:                             9163.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    107.5447      4.109     26.174      0.0

In [ ]:
# The first model has a lower MSE (1694.69), meaning the predictions are generally more accurate.
# The second model has a higher MSE (1725.67), indicating slightly worse prediction accuracy.
# Since the goal is to minimize the MSE, the first model performs better.
# Removing the variables 'Survived' and 'Age' slightly worsens the model's performance.
# The first model, which includes these variables, has better predictive accuracy (lower MSE) and a better fit (higher R-squared, lower AIC/BIC).
#Therefore, removing 'Survived' and 'Age' does not improve the model performance, and it's better to keep them in the model.